# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [84]:
df_curva.iloc[0,1]

0.0008111102098831921

In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [31]:
c_p = hw.CallPutt.CALL
c_p = hw.CallPut.PUT

AttributeError: module 'modules.hull_white' has no attribute 'CallPutt'

Ver toda la documentación:

In [32]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


In [44]:
strike=0.984
t0=0
t1=1
t2=2
gamma=1
sigma=0.05
hw.zcb_call_put(hw.CallPut.CALL,strike,t1,t2,curva(0.0001),math.exp(-curva(365)),math.exp(-curva(730) * 730 / 365.0),gamma,sigma)

0.018233069228648313

## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 1282 y 2454 días.


Pregunta 2:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,049.00 (365 días después).


Pregunta 3:

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.


Pregunta 4:

Considere un swap a 1Y con cupones semestrales de 1.6% (lineal). Si la tasa
cupón cero a 0.5Y es 1.5% (exp), calcule la tasa cero a 1Y.


Pregunta 5:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.4%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 6:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 7:

Calcule un paso de simulac

## Respuestas

### Pregunta 1

In [12]:
crv = interp1d(df_curva['plazo'], df_curva['tasa'],'linear')
t1=1282
t2=2454
r1=crv(t1)
r2=crv(t2)
df1=math.exp(-r1* t1 / 365)
df2=math.exp(-r2* t2 / 365)
forward= (df1/df2-1.0)* 360 / (t2-t1)
print("la tasa forward es igual a" ,forward)

la tasa forward es igual a 0.0059331987373331795


### Pregunta 2

In [14]:
icp0=10000
icp365=10049
TNA = (icp365/icp0 - 1)* (360 / 365)
print("la tasa nominal anual (TNA) es igual a",TNA)

la tasa nominal anual (TNA) es igual a 0.0048328767123286724


### Pregunta 3

En sí la formula para valuar una opción es la misma y es exp(A-B*r), sin embargo la composición de A son distintas.

### Pregunta 4

In [22]:
L=(1+0.016*0.5)+(1+0.016*1)
print(L)
exp=math.exp(0.015*0.5)
print(exp)
A=L-exp
tasa=math.log(A)
print("La tasa cero cupón para 1Y debiese ser",tasa)
## esto lo hice igualando la suma de los dos factores de capitalización
#para ambos periodos y así despejar la tasa exp que faltaba a 1Y


2.024
1.0075281954445339
La tasa cero cupón para 1Y debiese ser 0.016337615930257902


### Pregunta 5

In [46]:
strike=0.984
t0=0
t1=1
t2=2
gamma=1
sigma=0.005
hw.zcb_call_put(hw.CallPut.CALL,strike,t1,t2,curva(0.0001),
                math.exp(-curva(365)),
                math.exp(-curva(730) * 730 / 365.0)
                ,gamma,sigma)


0.015556384324646233

### Pregunta 6

La gran diferencia que existe entre valorizar con HW simulaciones versus medida T-Forward es que en la última el drift es
modificado, agregando un movimiento al thetha. especificamente el nuevo thetha sigue la siguiente forma 
theta_barra= thetha - B*sigma

### Pregunta 7

In [92]:
gamma=1
sigma=0.005
dt=1/264
r0=df_curva.iloc[0,1]
r_=r0
n_aleatorio=1.6027

r1=r_*gamma*dt + (1-gamma*dt)*r0 + sigma*math.sqrt(dt)*n_aleatorio
print("La tasa incial en el tiempo 0 extraida de la curva cero cupón es ",r0)
print("La tasa simulada un paso más adelante es igual a ",r1)


La tasa incial en el tiempo 0 extraida de la curva cero cupón es  0.0008111102098831921
La tasa simulada un paso más adelante es igual a  0.0013043070413646335
